In [1]:
%matplotlib inline
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
import cv2

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
n_samples = 5000
rescaled_dim = 32

path = '/Users/theisenm'
df = pd.read_csv(path + '/Dropbox/Kaggle/train.csv')
sample = pd.read_csv(path + '/Dropbox/Kaggle/sample_submission.csv')
lb = MultiLabelBinarizer()


In [3]:
def set_output(df):
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    return y

def set_input(train_features):
    X = np.squeeze(np.array(train_features, np.float16) / 255.)
    return X

In [4]:
# FEATURE EXTRACTION

# Raw features
raw_features_train = [cv2.resize(cv2.imread(path + '/Kaggle/train-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)) for name in df.head(n_samples)['image_name'].values]
#raw_features_test = [cv2.resize(plt.imread(path + '/Kaggle/test-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)).reshape(1, -1) for name in sample['image_name'].values]

In [5]:
# SET X's & Y's
y = set_output(df)
X = set_input(raw_features_train)
#X_sub = set_input(raw_features_test)

print(X.shape, y.shape, lb.classes_)
#print X_sub.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train.shape

((5000, 32, 32, 3), (5000, 17), array(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming',
       'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation',
       'habitation', 'haze', 'partly_cloudy', 'primary', 'road',
       'selective_logging', 'slash_burn', 'water'], dtype=object))


(3350, 32, 32, 3)

In [6]:
def myGenerator():
    yield X_train, y_train

In [9]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(17, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [11]:
generator = myGenerator()

# train the model on the new data for a few epochs
model.fit(X_train, y_train, batch_size=100, epochs=2)

#model.fit_generator(generator, 100, epochs=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=100, epochs=2)
#model.fit_generator(X_train, y_train, epochs=2)

Epoch 1/2
3350/3350 [==============================] - 3s - loss: nan     
Epoch 2/2
3350/3350 [==============================] - 3s - loss: nan     
(0, 'input_2')
(1, 'conv2d_95')
(2, 'batch_normalization_95')
(3, 'activation_95')
(4, 'conv2d_96')
(5, 'batch_normalization_96')
(6, 'activation_96')
(7, 'conv2d_97')
(8, 'batch_normalization_97')
(9, 'activation_97')
(10, 'max_pooling2d_5')
(11, 'conv2d_98')
(12, 'batch_normalization_98')
(13, 'activation_98')
(14, 'conv2d_99')
(15, 'batch_normalization_99')
(16, 'activation_99')
(17, 'max_pooling2d_6')
(18, 'conv2d_103')
(19, 'batch_normalization_103')
(20, 'activation_103')
(21, 'conv2d_101')
(22, 'conv2d_104')
(23, 'batch_normalization_101')
(24, 'batch_normalization_104')
(25, 'activation_101')
(26, 'activation_104')
(27, 'average_pooling2d_10')
(28, 'conv2d_100')
(29, 'conv2d_102')
(30, 'conv2d_105')
(31, 'conv2d_106')
(32, 'batch_normalization_100')
(33, 'batch_normalization_102')
(34, 'batch_normalization_105')
(35, 'batch_normal